In [1]:
from nltk.corpus import reuters
import nltk
nltk.download('reuters')
!unzip /root/nltk_data/corpora/reuters.zip -d /root/nltk_data/corpora

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\anant\AppData\Roaming\nltk_data...
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
from nltk.corpus import reuters
len(reuters.categories())

90

In [3]:
classes=['acq','corn','crude', 'earn', 'grain', 'interest', 'money-fx', 'ship', 'trade', 'wheat']
document_id_list=[]


In [4]:
for classs in classes:
  current_class_document_id_list=reuters.fileids(classs);
  document_id_list.extend(current_class_document_id_list)
distinct_document_id_list=list(set(document_id_list))

In [5]:
category_map={'acq':0,'corn':1,'crude':2, 'earn':3, 'grain':4, 'interest':5, 'money-fx':6, 'ship':7, 'trade':8, 'wheat':9}
processed_doc_ids=[]
processed_label_ids=[]

In [6]:
for i in range(len(distinct_document_id_list)):
  if len(reuters.categories(distinct_document_id_list[i]))==1:
    processed_label_ids.append(category_map[reuters.categories(distinct_document_id_list[i])[0]])
    processed_doc_ids.append(distinct_document_id_list[i])
  else:
    count=0
    for category in reuters.categories(distinct_document_id_list[i]):
      if category in classes:
        count+=1
        required_category=category
    if count == 1:  
      processed_doc_ids.append(distinct_document_id_list[i])
      processed_label_ids.append(category_map[required_category])

In [7]:
def verify_preprocessing(processed_doc_ids):
  sum=0
  for id in processed_doc_ids:
    count=0
    for category in reuters.categories(id):
        if category in classes:
          count+=1
    if count>1:
      print('Not working')

In [8]:
verify_preprocessing(processed_doc_ids)
print(len(processed_doc_ids))

8201


In [9]:
import nltk
nltk.download('punkt')
documents=reuters.paras(fileids=processed_doc_ids)
words_in_dataset=[]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anant\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
for j in range(len(documents)):
  words_in_current_document=documents[j]
  for i in range(len(words_in_current_document)):
    words_in_dataset.extend(words_in_current_document[i])

In [11]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
def stem_tokens(tokens):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))  
    return stemmed
document_ids_terms = {}

In [12]:
i=0
for document_id in processed_doc_ids: 
  document_ids_terms[document_id]=[]
  for para in reuters.paras(fileids=[document_id])[0]:
    document_ids_terms[document_id].extend(stem_tokens(para))
  i+=1
sum=0
for abc in document_ids_terms:
  sum+=len(abc)
print(sum)

100736


In [13]:
print(len(set(processed_doc_ids)))
print(len(processed_doc_ids))
print(len(processed_doc_ids))

8201
8201
8201


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_matrix = TfidfVectorizer(stop_words='english', input='content')
tfs = tfidf_matrix.fit_transform([" ".join(l) for l in document_ids_terms.values()])

In [15]:
tfs.shape

(8201, 19977)

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random as rd

In [17]:
class KMeans:

    def __init__(self, n_clusters,n_iter):
        self.data = pd.DataFrame()
        self.n_clusters = n_clusters
        self.centroids = pd.DataFrame()
        self.clusters = np.ndarray(1)
        self.old_centroids = pd.DataFrame()
        self.verbose = False
        self.predictions = list()
        self.n_iter=n_iter

    def train(self, df, verbose):
        self.verbose = verbose
        self.data = df.copy(deep=True)
        self.clusters = np.zeros(len(self.data))
        if 'species' in self.data.columns:
            self.data.drop('species', axis=1, inplace=True)

        unique_rows = self.data.drop_duplicates()
        unique_rows.reset_index(drop=True, inplace=True)
        self.centroids = unique_rows.sample(n=self.n_clusters)
        self.centroids.reset_index(drop=True, inplace=True)
        if self.verbose:
            print("\nRandomly initiated centroids:")
            print(self.centroids)

        self.old_centroids = pd.DataFrame(np.zeros(shape=(self.n_clusters, self.data.shape[1])),
                                          columns=self.data.columns)
        while not self.old_centroids.equals(self.centroids) and self.n_iter!=0:
            if self.verbose:
                time.sleep(3)

            self.old_centroids = self.centroids.copy(deep=True)

            for row_i in range(0, len(self.data)):
                distances = list()
                point = self.data.iloc[row_i]

                for row_c in range(0, len(self.centroids)):
                    centroid = self.centroids.iloc[row_c]
                    distances.append(np.linalg.norm(point - centroid))

                self.clusters[row_i] = np.argmin(distances)
            for cls in range(0, self.n_clusters):

                cls_idx = np.where(self.clusters == cls)[0]

                if len(cls_idx) == 0:
                    self.centroids.loc[cls] = self.old_centroids.loc[cls]
                else:
                    self.centroids.loc[cls] = self.data.iloc[cls_idx].mean()
                    
                if self.verbose:
                    print("\nRow indices belonging to cluster {}: [n={}]".format(cls, len(cls_idx)))
                    print(cls_idx)

            if self.verbose:
                print("\nOld centroids:")
                print(self.old_centroids)
                print("New centroids:")
                print(self.centroids)
            self.n_iter-=1

In [18]:
number_of_clusters = 10
kmeans = KMeans(n_clusters=number_of_clusters,n_iter=100)
kmeans.train(df=pd.DataFrame(tfs.toarray()), verbose=False)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils.linear_assignment_ import linear_assignment
def _make_cost_m(cm):
    s = np.max(cm)
    return (- cm + s)
def align_predicted_and_correct_cluster_labesl(correct_clusters,predicted_clusters):
  cm=confusion_matrix(correct_clusters,predicted_clusters)
  print(cm)
  print(np.trace(cm) / np.sum(cm))
  indexes = linear_assignment(_make_cost_m(cm))
  js = [e[1] for e in sorted(indexes, key=lambda x: x[0])]
  cm2 = cm[:, js]
  print(cm2)
  print(np.trace(cm2) / np.sum(cm2))
  return indexes

In [ ]:
indexes=align_predicted_and_correct_cluster_labesl(processed_label_ids,kmeans.labels_)


In [ ]:
cfnmatrix=np.array([[2239,    0,    6,    1,   45,   16,   12,    0,    0,    0],
 [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
 [  66,    0,  380,    0,    5,    0,    6,    0,    1,    0],
 [ 165,  806,    4,  944,  865,   17,    2,  484,  132,  511],
 [  94,    0,    1,    0,    6,    0,   12,    0,    0,    0],
 [  16,   0,    1 ,   0,    1,  225,   41,   0,    6,    0],
 [  30,    0 ,   0,    0,    5,  103,  349,    0,   12,    0],
 [ 156,    0,    2,    0,    3,    0,    5,    0,    0,    0],
 [  19,    0,    3,    0,   21,    1,  311,    0,   70,    0],
 [   1,   0,    0 ,   0,    0,    0,    0,    0,    0,    0],])

In [ ]:
row_sum=0
column_sum=0
for j in range(10):
  row_sum+=cfnmatrix[0][j]
print(row_sum)
for j in range(10):
  column_sum+=cfnmatrix[j][0]
column_sum-=cfnmatrix[0][0]
print(column_sum)
cfnmatrix.sum()-row_sum-column_sum

In [ ]:
def TP_TN_FP_FN(confusion_matrix,n):
  table=pd.DataFrame(columns=['Class','True Positive','True Negative','False Positive','False Negative'])
  for i in range(10):
    total_sum=np.sum(confusion_matrix)
    true_positive=confusion_matrix[i][i]
    false_postive=0
    for j in range(10):
      if j!=i:
        false_postive+=confusion_matrix[j][i]
    false_negative=0
    for j in range(10):
      if j!=i:
        false_negative+=confusion_matrix[i][j]
    row_sum=0
    column_sum=0
    for j in range(10):
      row_sum+=confusion_matrix[i][j]
    for j in range(10):
      column_sum+=confusion_matrix[j][i]
    column_sum-=confusion_matrix[i][i]
    true_negative=total_sum-row_sum-column_sum
    
    table.loc[len(table.index)] = [i,true_positive,true_negative,false_postive,false_negative]
  return table

In [ ]:
dataframe=TP_TN_FP_FN(cfnmatrix,10)
dataframe

In [ ]:
transformation_dictionary={}
for abc in indexes:
  transformation_dictionary[abc[0]]=abc[1]

In [ ]:
aligned_label_ids=[]
for processed_label_id in processed_label_ids:
  processed_label_id
  aligned_label_ids.append(transformation_dictionary[processed_label_id])

In [ ]:
matrix = classification_report(aligned_label_ids,kmeans.labels_)
print('Classification report : \n',matrix)

In [ ]:
import numpy as np
from sklearn import metrics
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
def show_metrics(correct_clusters,predicted_clusters):
    contingency_matrix = metrics.cluster.contingency_matrix(correct_clusters, predicted_clusters)
    purity_score= np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)
    print('Purity Score',purity_score)
    print('Normalized Mutual Information',normalized_mutual_info_score(correct_clusters,predicted_clusters))
    print('Rand Index',adjusted_rand_score(correct_clusters,predicted_clusters))